In [22]:
import clr, os, winreg
from itertools import islice

# This boilerplate requires the 'pythonnet' module.
# The following instructions are for installing the 'pythonnet' module via pip:
#    1. Ensure you are running a Python version compatible with PythonNET. Check the article "ZOS-API using Python.NET" or
#    "Getting started with Python" in our knowledge base for more details.
#    2. Install 'pythonnet' from pip via a command prompt (type 'cmd' from the start menu or press Windows + R and type 'cmd' then enter)
#
#        python -m pip install pythonnet

# determine the Zemax working directory
aKey = winreg.OpenKey(winreg.ConnectRegistry(None, winreg.HKEY_CURRENT_USER), r"Software\Zemax", 0, winreg.KEY_READ)
zemaxData = winreg.QueryValueEx(aKey, 'ZemaxRoot')
NetHelper = os.path.join(os.sep, zemaxData[0], r'ZOS-API\Libraries\ZOSAPI_NetHelper.dll')
winreg.CloseKey(aKey)

# add the NetHelper DLL for locating the OpticStudio install folder
clr.AddReference(NetHelper)
import ZOSAPI_NetHelper

pathToInstall = ''
# uncomment the following line to use a specific instance of the ZOS-API assemblies
#pathToInstall = r'C:\C:\Program Files\Zemax OpticStudio'

# connect to OpticStudio
success = ZOSAPI_NetHelper.ZOSAPI_Initializer.Initialize(pathToInstall);

zemaxDir = ''
if success:
    zemaxDir = ZOSAPI_NetHelper.ZOSAPI_Initializer.GetZemaxDirectory();
    print('Found OpticStudio at:   %s' + zemaxDir);
else:
    raise Exception('Cannot find OpticStudio')

# load the ZOS-API assemblies
clr.AddReference(os.path.join(os.sep, zemaxDir, r'ZOSAPI.dll'))
clr.AddReference(os.path.join(os.sep, zemaxDir, r'ZOSAPI_Interfaces.dll'))
import ZOSAPI

TheConnection = ZOSAPI.ZOSAPI_Connection()
if TheConnection is None:
    raise Exception("Unable to intialize NET connection to ZOSAPI")

TheApplication = TheConnection.ConnectAsExtension(0)
if TheApplication is None:
    raise Exception("Unable to acquire ZOSAPI application")

if TheApplication.IsValidLicenseForAPI == False:
    raise Exception("License is not valid for ZOSAPI use.  Make sure you have enabled 'Programming > Interactive Extension' from the OpticStudio GUI.")

TheSystem = TheApplication.PrimarySystem
if TheSystem is None:
    raise Exception("Unable to acquire Primary system")

def reshape(data, x, y, transpose = False):
    """Converts a System.Double[,] to a 2D list for plotting or post processing
    
    Parameters
    ----------
    data      : System.Double[,] data directly from ZOS-API 
    x         : x width of new 2D list [use var.GetLength(0) for dimension]
    y         : y width of new 2D list [use var.GetLength(1) for dimension]
    transpose : transposes data; needed for some multi-dimensional line series data
    
    Returns
    -------
    res       : 2D list; can be directly used with Matplotlib or converted to
                a numpy array using numpy.asarray(res)
    """
    if type(data) is not list:
        data = list(data)
    var_lst = [y] * x;
    it = iter(data)
    res = [list(islice(it, i)) for i in var_lst]
    if transpose:
        return self.transpose(res);
    return res
    
def transpose(data):
    """Transposes a 2D list (Python3.x or greater).  
    
    Useful for converting mutli-dimensional line series (i.e. FFT PSF)
    
    Parameters
    ----------
    data      : Python native list (if using System.Data[,] object reshape first)    
    
    Returns
    -------
    res       : transposed 2D list
    """
    if type(data) is not list:
        data = list(data)
    return list(map(list, zip(*data)))

print('Connected to OpticStudio')

# The connection should now be ready to use.  For example:
print('Serial #: ', TheApplication.SerialCode)

# Insert Code Here

import numpy as np
import matplotlib.pyplot as plt
import random
import shutil
from enum import Enum


Found OpticStudio at:   %sc:\program files\ansys zemax opticstudio 2023 r2.00
Connected to OpticStudio
Serial #:  A1154551


In [6]:
path = 'D:/AI_Alignment/data/data1/error.txt'
with open(path,'r',encoding='utf-8') as file :
    lines = file.readlines()
    
error = []
for i in range(len(lines)) :
    a = lines[i].strip()
    error.append(float(a))
    
error

[0.092, 0.198, 0.333, -0.023, -0.125, -0.067]

In [7]:
surf11 = TheSystem.LDE.GetSurfaceAt(11)
surf11.Thickness = error[2]
surf10 = TheSystem.LDE.GetSurfaceAt(10)
surf10.GetCellAt(12).DoubleValue = error[0]
surf10.GetCellAt(13).DoubleValue = error[1]
surf10.GetCellAt(14).DoubleValue = error[3]
surf10.GetCellAt(15).DoubleValue = error[4]
surf10.GetCellAt(16).DoubleValue = error[5]

In [14]:
zern = TheSystem.Analyses.New_ZernikeFringeCoefficients()
zern_setting = zern.GetSettings()
#zern_setting = ZOSAPI.Analysis.Settings.IAS_ZernikeFringeCoefficients(zern_setting)
#zern_setting.Field.SetFieldNumber(2)
zern.ApplyAndWaitForCompletion()
zern_results = zern.GetResults()
zern_results.GetTextFile('D:/AI_Alignment/zernike.txt') #Result file

True

In [25]:
TheMFE = TheSystem.MFE

MaxTerm = 37


ZERNType = ZOSAPI.Editors.MFE.MeritOperandType.ZERN


FirstOperand = TheMFE.GetOperandAt(1)


FirstOperand.ChangeType(ZERNType)


Wave = 1
Samp = 2
Field = 1
Type = 1 # Standard coefficients
Epsilon = 0.0
Vertex = 0

FirstOperand.GetOperandCell(2).DoubleValue = MaxTerm     # Term
FirstOperand.GetOperandCell(3).DoubleValue = Wave        # Wave
FirstOperand.GetOperandCell(4).DoubleValue = Samp        # Samp
FirstOperand.GetOperandCell(5).DoubleValue = Field       # Field
FirstOperand.GetOperandCell(6).DoubleValue = Type        # Type
FirstOperand.GetOperandCell(7).DoubleValue = Epsilon      # Epsilon
FirstOperand.GetOperandCell(8).DoubleValue = Vertex      # Vertex?


for Term in range(1, MaxTerm):

    NewOperand = TheMFE.InsertNewOperandAt(2)


    NewOperand.ChangeType(ZERNType)

    
    # Adjust its settings
    NewOperand.GetOperandCell(2).DoubleValue = Term        # Term
    NewOperand.GetOperandCell(3).DoubleValue = Wave        # Wave
    NewOperand.GetOperandCell(4).DoubleValue = Samp        # Samp
    NewOperand.GetOperandCell(5).DoubleValue = Field       # Field
    NewOperand.GetOperandCell(6).DoubleValue = Type        # Type
    NewOperand.GetOperandCell(7).DoubleValue = Epsilon      # Epsilon
    NewOperand.GetOperandCell(8).DoubleValue = Vertex      # Vertex?


TheMFE.CalculateMeritFunction()


for Index in range(MaxTerm, 0, -1):

    CurrentOperand = TheMFE.GetOperandAt(Index)
    Term = CurrentOperand.GetOperandCell(2).IntegerValue
    TermValue = CurrentOperand.Value
    print('Z {0} \t {1: .8f}'.format(Term, TermValue))
    

RemotingException: IPC 포트에 쓰지 못했습니다. 파이프가 닫히는 중입니다.


Server stack trace: 
   위치: System.Runtime.Remoting.Channels.Ipc.IpcPort.Write(Byte[] data, Int32 offset, Int32 size)
   위치: System.Runtime.Remoting.Channels.ChunkedMemoryStream.WriteTo(Stream stream)
   위치: System.Runtime.Remoting.Channels.Ipc.IpcClientHandler.SendRequest(IMessage msg, ITransportHeaders headers, Stream contentStream)
   위치: System.Runtime.Remoting.Channels.Ipc.IpcClientTransportSink.ProcessMessage(IMessage msg, ITransportHeaders requestHeaders, Stream requestStream, ITransportHeaders& responseHeaders, Stream& responseStream)
   위치: System.Runtime.Remoting.Channels.BinaryClientFormatterSink.SyncProcessMessage(IMessage msg)

Exception rethrown at [0]: 
   위치: System.Runtime.Remoting.Proxies.RealProxy.HandleReturnMessage(IMessage reqMsg, IMessage retMsg)
   위치: System.Runtime.Remoting.Proxies.RealProxy.PrivateInvoke(MessageData& msgData, Int32 type)
   위치: ZOSAPI.IOpticalSystem.get_MFE()